# Abstract

# Import Functions

In [26]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [27]:
import os
import sys

# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath('../')

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the desired function or class
from Functions.seaborn_plotting_function import seaborn_plotter
from Functions.basic_functions import save_pickle
#from Functions.data_transformation_pipeline import Transformation_Pipeline

# Logistics

In [28]:
raw_database_folder = '../Database/raw'
mart_database_folder = '../Database/marts'

In [29]:
plotter = seaborn_plotter()

# load files

In [30]:
raw_db = {}

In [31]:

for file in os.listdir(raw_database_folder):
    if '.csv' in file:
        key = file.replace('.csv', '')
        raw_db[key] = pd.read_csv(os.path.join(raw_database_folder, file))
        raw_db[key] = raw_db[key].convert_dtypes()

raw_db['contact_log']['date'] = pd.to_datetime(raw_db['contact_log']['date'])
raw_db['members']['member_join_date'] = pd.to_datetime(raw_db['members']['member_join_date'])

In [32]:
for key in raw_db:
    print(key)

contact_log
members
member_contacts
type_of_contact


# Step 1: ETL process (would want to move this to DBT or something of that sort to do an ETL process using an orchestration tool)

In [33]:
member_contact_columns = ['connection_id', 'member_id', 'contact_id', 'name', 'contact_type', 'family', 'local']
contact_member_logs_columns = ['connection_id', 'member_id', 'member_name', 'contact_id', 'contact_name', 'contact_type', 'family', 'local',
                               'most_recent_contact', 'type_of_contact', 'contact_label', 'days_since_contact', 'number_of_contacts']
contact_log_columns = ['member_id', 'contact_id', 'date', 'type_of_contact']
members_columns = ['member_id', 'member_join_date']

## Step 1A: get connection id most recent contact

In [34]:
raw_db['contact_log']['most_recent_contact'] = raw_db['contact_log'].groupby(['member_id', 'contact_id'])['date'].rank(method='max', ascending=False)
raw_db['contact_log']['most_recent_contact'] = raw_db['contact_log']['most_recent_contact'].astype(int)
recent_contacts = raw_db['contact_log'][raw_db['contact_log']['most_recent_contact'] == 1]

## Step 1B: get contact_member_log

In [35]:
contact_member_logs = pd.merge(raw_db['member_contacts'].filter(member_contact_columns), recent_contacts.filter(contact_log_columns), how='inner', on = ['member_id', 'contact_id'])
contact_member_logs = contact_member_logs.rename(columns= {'date': 'most_recent_contact'})

In [36]:
no_contact_yet = raw_db['member_contacts'][~raw_db['member_contacts']['connection_id'].isin(contact_member_logs['connection_id'].unique())]
no_contact_yet = no_contact_yet.filter(member_contact_columns)

In [37]:
no_contact_yet = pd.merge(no_contact_yet, raw_db['members'].filter(members_columns), how='inner', on='member_id')
no_contact_yet['type_of_contact'] = 0
no_contact_yet = no_contact_yet.rename(columns= {'member_join_date': 'most_recent_contact'})

## Step 1C: Additional Metrics

In [38]:
number_of_contacts = raw_db['contact_log'].groupby(['member_id', 'contact_id']).count()
number_of_contacts = number_of_contacts.filter(['date'])
number_of_contacts = number_of_contacts.rename(columns = {'date': 'number_of_contacts'})
number_of_contacts = number_of_contacts.reset_index(drop = False)

In [39]:
contact_member_logs = pd.concat([contact_member_logs, no_contact_yet])
contact_member_logs['days_since_contact'] = dt.date.today()
contact_member_logs['days_since_contact'] = pd.to_datetime(contact_member_logs['days_since_contact'])
contact_member_logs['days_since_contact'] = (contact_member_logs['days_since_contact'] - contact_member_logs['most_recent_contact']).dt.days

In [40]:
contact_member_logs = pd.merge(contact_member_logs, number_of_contacts, how = 'left', on = ['member_id', 'contact_id'])
contact_member_logs = contact_member_logs.fillna(0)
contact_member_logs['number_of_contacts'] = contact_member_logs['number_of_contacts'].astype(int)
contact_member_logs['type_of_contact'] = contact_member_logs['type_of_contact'].astype(int)
contact_member_logs

,connection_id,member_id,contact_id,name,contact_type,family,local,most_recent_contact,type_of_contact,days_since_contact,number_of_contacts
0,22,1,22,Gustavo Salazar,Individual,0,0,2025-06-26,1,208,2
1,34,1,34,Gabriel Wong,Individual,0,1,2025-06-24,4,210,1
2,35,1,35,Tiffany Sha,Individual,0,1,2025-06-24,4,210,1
3,52,1,52,Vincent Tsai,Individual,0,0,2025-06-26,1,208,1
4,1,1,1,Sammy Wang,Individual,1,0,2025-06-10,0,224,0
...,...,...,...,...,...,...,...,...,...,...,...
80,81,1,81,David Chang,Individual,0,1,2025-06-10,0,224,0
81,82,1,82,Peter Chan,Individual,0,1,2025-06-10,0,224,0
82,83,1,83,Jenny Trang,Individual,0,0,2025-06-10,0,224,0
83,84,1,84,Ben kariya,Individual,0,0,2025-06-10,0,224,0


In [41]:
contact_member_logs = contact_member_logs.rename(columns={'name': 'contact_name'})
contact_member_logs = pd.merge(contact_member_logs, raw_db['members'].filter(['member_id', 'member_name']), how='inner', on='member_id')
contact_member_logs = pd.merge(contact_member_logs, raw_db['type_of_contact'], how='inner', on='type_of_contact')
contact_member_logs = contact_member_logs.filter(contact_member_logs_columns)
contact_member_logs

,connection_id,member_id,member_name,contact_id,contact_name,contact_type,family,local,most_recent_contact,type_of_contact,contact_label,days_since_contact,number_of_contacts
0,22,1,Albert Sze,22,Gustavo Salazar,Individual,0,0,2025-06-26,1,text,208,2
1,34,1,Albert Sze,34,Gabriel Wong,Individual,0,1,2025-06-24,4,in person meeting,210,1
2,35,1,Albert Sze,35,Tiffany Sha,Individual,0,1,2025-06-24,4,in person meeting,210,1
3,52,1,Albert Sze,52,Vincent Tsai,Individual,0,0,2025-06-26,1,text,208,1
4,1,1,Albert Sze,1,Sammy Wang,Individual,1,0,2025-06-10,0,<NA>,224,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,81,1,Albert Sze,81,David Chang,Individual,0,1,2025-06-10,0,<NA>,224,0
81,82,1,Albert Sze,82,Peter Chan,Individual,0,1,2025-06-10,0,<NA>,224,0
82,83,1,Albert Sze,83,Jenny Trang,Individual,0,0,2025-06-10,0,<NA>,224,0
83,84,1,Albert Sze,84,Ben kariya,Individual,0,0,2025-06-10,0,<NA>,224,0


In [42]:
contact_member_logs.to_csv(os.path.join(mart_database_folder, 'contact_member_logs.csv'), index=False)

# Step 2: normalize data for local and non local data

In [43]:
normalize_columns = ['type_of_contact', 'days_since_contact', 'number_of_contacts']

In [44]:
etl_data = {'local': contact_member_logs[contact_member_logs['local'] == 1],
            'non_local': contact_member_logs[contact_member_logs['local'] == 0],
           }

In [45]:
for data_type in ('local', 'non_local'):
    for column_name in normalize_columns:
        if column_name == 'type_of_contact':
            etl_data[data_type][f"norm_{column_name}"] = etl_data[data_type][column_name] / raw_db['type_of_contact']['type_of_contact'].max()
        else:
            scaler = MinMaxScaler()
            etl_data[data_type][f"norm_{column_name}"] = scaler.fit_transform(etl_data[data_type][[column_name]].values)

    for column_name in normalize_columns:
        if column_name != 'days_since_contact':
            etl_data[data_type][f"inv_norm_{column_name}"] = 1 - etl_data[data_type][f"norm_{column_name}"]
    etl_data[data_type]['score'] = etl_data[data_type][['inv_norm_type_of_contact', 'norm_days_since_contact', 'inv_norm_number_of_contacts']].mean(axis=1)


C:\Users\alber\AppData\Local\Temp\ipykernel_7936\1294068315.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etl_data[data_type][f"norm_{column_name}"] = etl_data[data_type][column_name] / raw_db['type_of_contact']['type_of_contact'].max()
C:\Users\alber\AppData\Local\Temp\ipykernel_7936\1294068315.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etl_data[data_type][f"norm_{column_name}"] = scaler.fit_transform(etl_data[data_type][[column_name]].values)
C:\Users\alber\AppData\Local\Temp\ipykernel_7936

# Step 3: Selection

In [46]:
def simple_binary_search_closest(arr: list, target: float) -> int:
    if not arr:
        return -1 # Return -1 for an empty list

    low = 0
    high = len(arr) - 1
    closest_idx = 0 # Initialize with first index as a default closest

    while low <= high:
        mid = (low + high) // 2
        mid_val = arr[mid]

        # If we found an exact match
        if mid_val == target:
            return mid

        # Update closest_idx if current mid_val is closer
        if abs(mid_val - target) < abs(arr[closest_idx] - target):
            closest_idx = mid
        elif abs(mid_val - target) == abs(arr[closest_idx] - target):
            # If equally close, prioritize smaller index or other criteria
            # For this simple version, we'll keep the one we already have unless explicitly told otherwise
            pass

        # Adjust search range
        if mid_val < target:
            low = mid + 1
        else:
            high = mid - 1

    # Compare `arr[high]` (value just less than target) and `arr[low]` (value just greater than target)
    if low >= len(arr): # Target is greater than all elements
        return len(arr) - 1
    if high < 0: # Target is less than all elements
        return 0

    val_at_low = arr[low]
    val_at_high = arr[high] # high will be one less than low if no exact match found

    diff_low = abs(val_at_low - target)
    diff_high = abs(val_at_high - target)

    if diff_low <= diff_high:
        return low
    else:
        return high


In [47]:
for data_type in ('local', 'non_local'):
    percent_per_score = 100.00 / etl_data[data_type]['score'].sum()
    etl_data[data_type]['percent_cut'] = etl_data[data_type]['score'] * percent_per_score

    etl_data[data_type] = etl_data[data_type].reset_index(drop = True)
    etl_data[data_type]['cum_sum'] = etl_data[data_type]['percent_cut'].cumsum()

C:\Users\alber\AppData\Local\Temp\ipykernel_7936\1249357792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etl_data[data_type]['percent_cut'] = etl_data[data_type]['score'] * percent_per_score
C:\Users\alber\AppData\Local\Temp\ipykernel_7936\1249357792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etl_data[data_type]['percent_cut'] = etl_data[data_type]['score'] * percent_per_score


In [48]:
todays_suggestions_log = pd.DataFrame()
for data_type in ('local', 'non_local'):
    random_selection = np.random.rand() * 100
    idx_selection = simple_binary_search_closest(etl_data[data_type]['cum_sum'].tolist(), random_selection)

    todays_suggestions_log = pd.concat([todays_suggestions_log, etl_data[data_type][etl_data[data_type].index == idx_selection]])

todays_suggestions_log['record_date'] = dt.datetime.today().date()
todays_suggestions_log

,connection_id,member_id,member_name,contact_id,contact_name,contact_type,family,local,most_recent_contact,type_of_contact,...,number_of_contacts,norm_type_of_contact,norm_days_since_contact,norm_number_of_contacts,inv_norm_type_of_contact,inv_norm_number_of_contacts,score,percent_cut,cum_sum,record_date
8,46,1,Albert Sze,46,JG Crew,Group,0,1,2025-06-10,0,...,0,0.0,1.0,0.0,1.0,1.0,1.0,7.142857,50.000000,2026-01-20
58,69,1,Albert Sze,69,Craigie St Crew,Group,0,0,2025-06-10,0,...,0,0.0,1.0,0.0,1.0,1.0,1.0,1.477833,85.221675,2026-01-20


In [49]:
todays_suggestions_log.to_csv(os.path.join(mart_database_folder, 'selections.csv'), index=False)

In [50]:
for idx, row in todays_suggestions_log.iterrows():
    if row['local']:
        print(f'local: {raw_db["member_contacts"][raw_db["member_contacts"]["contact_id"] == row["contact_id"]]["name"].iloc[0]}')
    else:
        print(f'non-local: {raw_db["member_contacts"][raw_db["member_contacts"]["contact_id"] == row["contact_id"]]["name"].iloc[0]}')

local: JG Crew
non-local: Craigie St Crew
